In [ ]:
# ✅ 1. Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. Install dependencies
!pip install ultralytics
!pip install deep_sort_realtime
!pip install opencv-python-headless

In [ ]:
# ✅ Clean version without `.feature` usage
# ✅ Uses YOLOv8 + DeepSORT + IoU-based global ID management

import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import matplotlib.pyplot as plt
from IPython.display import clear_output

# ✅ Load Model
model = YOLO('/content/drive/MyDrive/Assignment Materials/Assignment Materials/best.pt')

# ✅ Setup DeepSORT
tracker = DeepSort(
    max_age=40,
    n_init=8,
    max_cosine_distance=0.2,
    nn_budget=200
)

# ✅ Video Paths
input_path = '/content/drive/MyDrive/Assignment Materials/Assignment Materials/15sec_input_720p.mp4'
output_path = '/content/drive/MyDrive/Assignment Materials/Assignment Materials/Result/output video.mp4'

cap = cv2.VideoCapture(input_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width, height = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# ✅ Global ID memory
next_global_id = 1
deep_to_global = {}
global_box_memory = {}

def iou(box1, box2):
    xa = max(box1[0], box2[0])
    ya = max(box1[1], box2[1])
    xb = min(box1[2], box2[2])
    yb = min(box1[3], box2[3])
    inter_area = max(0, xb - xa) * max(0, yb - ya)
    box1_area = (box1[2]-box1[0]) * (box1[3]-box1[1])
    box2_area = (box2[2]-box2[0]) * (box2[3]-box2[1])
    union = box1_area + box2_area - inter_area
    return inter_area / union if union else 0

def match_by_iou(current_box, threshold=0.4):
    best_match = None
    max_iou_score = threshold
    for gid, past_box in global_box_memory.items():
        score = iou(current_box, past_box)
        if score > max_iou_score:
            best_match = gid
            max_iou_score = score
    return best_match

frame_id = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    detections = []

    # Detect players with high confidence
    for box in results.boxes.data:
        x1, y1, x2, y2, conf, cls = box.tolist()
        if int(cls) == 2 and conf > 0.6:
            bbox = [x1, y1, x2 - x1, y2 - y1]
            detections.append((bbox, conf, "player"))

    # Track
    tracks = tracker.update_tracks(detections, frame=frame)
    used_ids = set()

    for track in tracks:
        if not track.is_confirmed() or track.time_since_update > 0:
            continue

        ltrb = list(map(int, track.to_ltrb()))
        deep_id = track.track_id

        # Global ID assignment
        if deep_id in deep_to_global:
            global_id = deep_to_global[deep_id]
        else:
            matched_gid = match_by_iou(ltrb)
            if matched_gid and matched_gid not in used_ids:
                global_id = matched_gid
            else:
                global_id = next_global_id
                next_global_id += 1
            deep_to_global[deep_id] = global_id

        # Save latest box for IoU match
        global_box_memory[global_id] = ltrb
        used_ids.add(global_id)

        # Draw box
        x1, y1, x2, y2 = ltrb
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, f'Player {global_id}', (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    # Preview
    if frame_id % 30 == 0:
        clear_output(wait=True)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plt.imshow(rgb)
        plt.title(f"Frame {frame_id}")
        plt.axis("off")
        plt.show()

    out.write(frame)
    frame_id += 1

cap.release()
out.release()
print("✅ Tracking complete! Output saved to:", output_path)
